## Getting tweets as old as 1 week ago

This script gets the tweets for a list of medications, as old as 1 week


https://dev.twitter.com/rest/public/search

"At this time, users represented by access tokens can make 180 requests/queries per 15 minutes. Using application-only auth, an application can make 450 queries/requests per 15 minutes on its own behalf without a user context."
"count = The number of tweets to return per page, up to a maximum of 100. Defaults to 15."
"The Search API is not complete index of all Tweets, but instead an index of recent Tweets. At the moment that index includes between 6-9 days of Tweets."

the results are just from the last 7 to 9 days

In [1]:
import sys
import tweepy
import datetime
import urllib
import signal
import json
from datetime import datetime, timedelta



In [2]:
#keys
consumer_key = " "; 
consumer_secret = " "; 
 
access_token = " "; 
access_token_secret = " "; 

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth_handler=auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)




In [9]:
class TweetFile:
    out = None
    first = True
    def start(self, name="full", type=".json"):
        fname = "data/tweets-"+ name + type
        self.out = open(fname,"w")
        self.out.write("[\n")
        self.first = True

    def end(self):
        if self.out is not None:
            self.out.write("\n]\n")
            self.out.close()
        self.out = None

    def write(self,tweet):
        if not self.first:
            self.out.write(",\n")
        self.first = False
        self.out.write(json.dumps(tweet._json).encode('utf8'))
        
    def write_text(self,text):
        if not self.first:
            self.out.write(",\n")
        self.first = False
        self.out.write(text.encode('utf8'))

In [26]:
def getMedication(medication, since_date_run, until_date_run):
    arg = medication  + " since:" + since_date_run + " " + "until:" + until_date_run
    print (arg)
    file_name = medication + "-" + since_date_run + "--" + until_date_run
    file = TweetFile()
    file.start(file_name, ".json")
    file_text = TweetFile()
    file_text.start(file_name, ".txt")
    number_of_tweets = 0
    last_date =  ""
    for tweet in tweepy.Cursor(api.search,q=arg,lang='en').items(750): 
        file.write(tweet)
        last_date = tweet._json['created_at']
        file_text.write_text("--------------------------------" + last_date)
        file_text.write_text(tweet._json['text'])
        number_of_tweets += 1
    file.end()
    file_text.end()
    return (number_of_tweets, last_date)
 
    

In [5]:
def get_medication_list(medications, since_date, until_date):
    medication_list = medications.split(",")

    until_date_run = until_date.strftime("%Y-%m-%d")
    since_date_run = since_date.strftime("%Y-%m-%d")

    # file to store summary inforamtion
   
    time_run = datetime.today().strftime("%H-%M-%S")
    summary_file = TweetFile()
    summary_file.start("summary_"+since_date_run + "--" + until_date_run+ "-" + time_run, ".json")

    for medication in medication_list:
        medication = medication.strip()
        number_of_tweets, last_date = getMedication (medication, since_date_run, until_date_run)
        if (number_of_tweets == 0):
            oldest_date = "null"
            
        else:
            date_object = datetime.strptime(last_date, '%a %b %d %X +0000 %Y')
            oldest_date = date_object.strftime("%Y-%m-%d")      
        
        summary = {}
        summary["medication"] = medication
        summary["oldest-date"] = oldest_date
        summary["since_date_run"] = since_date_run
        summary["until_date_run"] = until_date_run
        summary["number-of-tweets"] = number_of_tweets
        summary_json = json.dumps(summary, ensure_ascii=False)
        print  "medication: %s, oldest-date: %s, run-date: %s, number-of-tweets: %s" %(medication, oldest_date, until_date_run, number_of_tweets)
        summary_file.write_text(summary_json)
    summary_file.end()



In [24]:
    
until_date = datetime.today()
since_date = until_date - timedelta(days=9)

#medication lists 
medications = "humira,dronedarone,lamictal,pradaxa,paxil,zoledronic acid,trazodone,enbrel,cymbalta"
#medications = "quetiapine,cipro,lozenge"
#medications = "dabigatran,olanzapine,fluoxetine,vyvanse"
#medications = "seroquel,fosamax,paroxetine,effexor"
#medications = "nicotine,prozac,tysabri,rivaroxaban,baclofen,lamotrigine,venlafaxine"
#medications = "apixaban,avelox,levaquin,zyprexa,duloxetine"
#medications = "ofloxacin,geodon,victoza,metoprolol,viibryd,pristiq"
#medications = "nesina,factive"
#medications = "nesina,factive,gamma-aminobutyric acid,sabril"
#medications = "livalo,denosumab,bystolic,xarelto,floxin,boniva,saphris"
#medications = "ziprasidone,memantine,namenda,latuda,fycompa,canagliflozin,zometa,etanercept"
#medications = "lurasidone,alendronate,linagliptin,effient,vimpat,eliquis,liraglutide,pregabalin"
#medications = "onglyza,nicotrol inhaler,lyrica,invokana,commitlozenge,actonel,nicotrolinhaler,synthroid,albuterol"
#medications = "nasonex,spiriva,suboxone,nexium,januvia,valsartan,tamiflu"

get_medication_list(medications,since_date, until_date)


medication: humira, oldest-date: 2016-10-29, run-date: 2016-11-07, number-of-tweets: 263
medication:  dronedarone, oldest-date: 2016-11-06, run-date: 2016-11-07, number-of-tweets: 1
medication:  lamictal, oldest-date: 2016-10-29, run-date: 2016-11-07, number-of-tweets: 108
medication:  pradaxa, oldest-date: 2016-10-29, run-date: 2016-11-07, number-of-tweets: 28
medication:  paxil, oldest-date: 2016-10-29, run-date: 2016-11-07, number-of-tweets: 334
medication:  zoledronic acid, oldest-date: 2016-10-31, run-date: 2016-11-07, number-of-tweets: 14
medication:  trazodone, oldest-date: 2016-10-29, run-date: 2016-11-07, number-of-tweets: 132
medication:  enbrel, oldest-date: 2016-10-29, run-date: 2016-11-07, number-of-tweets: 172
medication:  cymbalta, oldest-date: 2016-10-29, run-date: 2016-11-07, number-of-tweets: 250


Notes:

* The following medications reached the limit of 750, which means that there are more tahn 750 tweets in the last 9 days:

    ** prozac, 
    
    ** nicotine, 
    
    ** lyrica 

    